In [ ]:
import numpy as np
import pandas as pd
import os
import seaborn as sns

In [ ]:
# looping over folders
directory = 'C:\\newaomixresult'
folders_list = []
for folder in sorted(os.listdir(directory)):

    f = os.path.join(directory, folder)
    f2 = os.path.join(f, f'{folder}_o-f-CDA')
    f3 = os.path.join(f2, 'AOMix-FO.txt')
    if os.path.isfile(f3):
        folders_list.append(f3)
    else:
        print(f3)
# 

In [ ]:
keys = []
values_frag1_ufo = []
values_frag2_ufo = []
values_pol = []
values_charge = []

folders_list_morethan2_frag_idxs = [21, 29, 59, 68]
for txt_file_idx, txt_file in enumerate(folders_list):
    if txt_file_idx not in folders_list_morethan2_frag_idxs:
        print(txt_file_idx)
        keys.append(txt_file)
        values_frag1_ufo.append(np.loadtxt(fname=txt_file, skiprows=52, usecols=1, max_rows=1))
        values_frag2_ufo.append(np.loadtxt(fname=txt_file, skiprows=52, usecols=2, max_rows=1))
        values_pol.append(np.loadtxt(fname=txt_file, skiprows=60, usecols=3, max_rows=1))
        values_charge.append(np.loadtxt(fname=txt_file, skiprows=61, usecols=8, max_rows=1))

frag1_ufos = dict(zip(keys, values_frag1_ufo))
frag2_ufos = dict(zip(keys, values_frag2_ufo))
pol = dict(zip(keys, values_pol))
charge = dict(zip(keys, values_charge))


frag1_ufos[folders_list[21]] = 20.2
frag1_ufos[folders_list[29]] = 13.7
frag1_ufos[folders_list[59]] = 22.0
frag1_ufos[folders_list[68]] = 54.3

frag2_ufos[folders_list[21]] = 37.6
frag2_ufos[folders_list[29]] = 8.2
frag2_ufos[folders_list[59]] = 37.2
frag2_ufos[folders_list[68]] = 13.6

pol[folders_list[21]] = 'NaN'
pol[folders_list[29]] = 'NaN'
pol[folders_list[59]] = 'NaN'
pol[folders_list[68]] = 'NaN'

charge[folders_list[21]] = 'NaN'
charge[folders_list[29]] = 'NaN'
charge[folders_list[59]] = 'NaN'
charge[folders_list[68]] = 'NaN'

In [ ]:
my_dict = {'file_path':list(frag1_ufos.keys()),
        'fragment_1_UFO':list(frag1_ufos.values()),
        'fragment_2_UFO':list(frag2_ufos.values()),
        'Polarization':list(pol.values()),
        'Charge_transfer':list(charge.values())}
df = pd.DataFrame(my_dict)

New Code 8.20.2023

In [ ]:
%matplotlib qt
import numpy as np
import pandas as pd
import os
from typing import List, TextIO

In [ ]:
filename = 'C:/codepracticeonlogfile/POPC36water.log'
if filename.endswith(".log"):
    reader: TextIO = open(filename, 'r')
    line_list: List[str] = reader.readlines()
    for line in reversed(line_list):
        if line.lstrip().__contains__('Normal termination of Gaussian'):
            result = True
            break
    reader.close()

In [ ]:
df = sns.load_dataset('iris')
sns.violinplot(x=df["species"], y=df["sepal_length"], palette='PiYG')

Extracting some Parameters from CDA files

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

In [ ]:
# looping over files in the folders
directory = 'C:\\newaomixresult'
folders_list = []
folders_list_log = []
for folder in sorted(os.listdir(directory)):
    f = os.path.join(directory, folder)
    f_log = os.path.join(f, f'{folder}.log')
    f2 = os.path.join(f, f'{folder}_o-f-CDA')
    f3 = os.path.join(f2, 'AOMix-FO.txt')
    if os.path.isfile(f_log):
        folders_list_log.append(f_log)
    if os.path.isfile(f3):
        folders_list.append(f3)
    else:
        print(f3)
# 

In [ ]:
# extracting some values from txt files
values_frag1_ufo = []; values_frag2_ufo = []; values_pol = []; values_charge = []
scf_energies = []; meyer_bond_orders = []; bde_evs = []; bde_kc_mols = []
sigma_dons = []; back_dons = []; homos = []; lufos_0 = []; lufos_1 = []
frag_2_mulliken_charges = []; frag_1_mulliken_charges = []

for txt_file, log_file in zip(folders_list[:10], folders_list_log[:10]):
    
    # extracting Fragment 1 UFO
    values_frag1_ufo.append(np.loadtxt(fname=txt_file, skiprows=52, usecols=1, max_rows=1))
    
    # extracting Fragment 2 UFO
    values_frag2_ufo.append(np.loadtxt(fname=txt_file, skiprows=52, usecols=2, max_rows=1))
    
    # extracting dPL
    values_pol.append(np.loadtxt(fname=txt_file, skiprows=60, usecols=3, max_rows=1))
    
    # extracting CT
    values_charge.append(np.loadtxt(fname=txt_file, skiprows=61, usecols=8, max_rows=1))
    
    # extracting SCF and homos and lufos_0 and lufos_1
    scfs_lines = []
    scf_idxs = []
    counter = 0
    sub_counter = 0
    with open(log_file, 'r') as f:
        for line in f:
            if "SCF Done:" in line:
                scfs_lines.append(line)
            if "Population analysis using the SCF Density." in line:
                scf_idxs.append(counter)
            if "Alpha  occ. eigenvalues --" in line and counter > 20000: # more probably true (tbc later)
                sub_counter += 1
            counter += 1 
    
    line_contain_pop_analysis_idx = scf_idxs[-1]
    scf_energies.append(np.array(scfs_lines[-1][25:39]).astype(float))

    homos.append(np.loadtxt(fname=log_file, skiprows=line_contain_pop_analysis_idx + sub_counter + 3,
                usecols=-1, max_rows=1))
    lufos_0.append(np.loadtxt(fname=log_file, skiprows=line_contain_pop_analysis_idx + sub_counter + 4,
                usecols=4, max_rows=1))
    lufos_1.append(np.loadtxt(fname=log_file, skiprows=line_contain_pop_analysis_idx + sub_counter + 4,
                usecols=5, max_rows=1))

    counter = 0
    with open(txt_file, 'r') as f:
        for line in f:
            if "ALPHA-SPIN ORBITALS" in line:
                line_idx_1 = counter
            if 'FO contributions (%) to all occupied MOs' in line:
                line_idx_2 = counter
            counter += 1
        
    # extracting meyer bond order
    meyer_bond_orders.append(np.loadtxt(fname=txt_file, skiprows=line_idx_1+6, usecols=1, max_rows=1))
    # extracting bde ev
    bde_evs.append(np.loadtxt(fname=txt_file, skiprows=line_idx_1-2, usecols=2, max_rows=1))
    # extracting bde kc
    bde_kc_mols.append(np.loadtxt(fname=txt_file, skiprows=line_idx_1-2, usecols=4, max_rows=1))
    # extracting sigma don
    sigma_dons.append(np.loadtxt(fname=txt_file, skiprows=line_idx_2+2, usecols=1, max_rows=1))
    # extracting back don
    back_dons.append(np.loadtxt(fname=txt_file, skiprows=line_idx_2+2, usecols=2, max_rows=1))
    # extracting Mulliken charges for fragment 2
    counter = 0
    with open(log_file, 'r') as f:
        for line in f:
            if "Mulliken charges:" in line and counter > 20000: # more probably true
                mc_line_idx = counter
            counter += 1 
    frag_2_vals = []
    proceed = True
    i = 0
    while proceed:
        h_or_o = np.loadtxt(fname=log_file, skiprows=mc_line_idx+2+i, usecols=1, max_rows=1, dtype='str')
        if not (h_or_o == np.array('O') or h_or_o == np.array('H')):
            proceed = False
            first_line_without_o_or_h = i
        frag_2_vals.append(np.loadtxt(fname=log_file, skiprows=mc_line_idx+2+i, usecols=2, max_rows=1, dtype=float)) 
        i += 1
    frag_2_mulliken_charges.append(np.sum(frag_2_vals[:-1]))
    
    
    # extracting Mulliken charges for fragment 1
    frag_1_vals = []
    proceed = True
    ii = 0
    while proceed:
        h_or_o = np.loadtxt(fname=log_file, skiprows=mc_line_idx+1+i+ii, usecols=1, max_rows=1, dtype='str')
        if h_or_o == 'of':
            proceed = False
            ii -= 1
        frag_1_vals.append(np.loadtxt(fname=log_file, skiprows=mc_line_idx+1+i+ii, usecols=2, max_rows=1, dtype=float)) 
        ii += 1
    frag_1_mulliken_charges.append(np.sum(frag_1_vals[:-1]))

In [ ]:
columns = ['IDs', 'SCF Energy', 'Meyer Bond Order', 'Sigma Don', 'Back Don',
            'Fragment 1 mulliken charge', 'Fragment 2 mulliken charge',
            'Bond Distance Energy (ev)', 'Bond Distance Energy (kc/mol)',
            'Homos', 'Lufo 0', 'Lufo 1', 'Fragment 1 UFO', 'Fragment 2 UFO',
            'dPL', 'CT']
data = [sorted(os.listdir(directory)), scf_energies, meyer_bond_orders, sigma_dons, back_dons,
        frag_1_mulliken_charges, frag_2_mulliken_charges, bde_evs, bde_kc_mols, homos, lufos_0,
        lufos_1, values_frag1_ufo, values_frag2_ufo, values_pol, values_charge]

data = np.array(data).T
df = pd.DataFrame(data, columns=columns, dtype=None)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(7,7))
ax.plot(df['IDs'].values, np.array(df['CT']).astype(float), marker='o')
ax.set_xticks(ticks=range(len(df['CT'].values)), labels=df['IDs'].values, rotation=45)
ax.set_yticks(ticks=np.linspace(0.03, 0.3, 20))

In [ ]:
keys = []
values_frag1_ufo = []
values_frag2_ufo = []
values_pol = []
values_charge = []
ct = []
n_waters = []
lipid_names = []
for txt_file_idx, txt_file in enumerate(folders_list):
    if txt_file_idx not in folders_list_morethan2_frag_idxs:
        n_water = [int(s) for s in txt_file[:27] if s.isdigit()]
        n_waters.append(n_water)
        lipid_names.append(txt_file[18:22])
        values_frag1_ufo.append(np.loadtxt(fname=txt_file, skiprows=52, usecols=1, max_rows=1))
        values_frag2_ufo.append(np.loadtxt(fname=txt_file, skiprows=52, usecols=2, max_rows=1))
        values_pol.append(np.loadtxt(fname=txt_file, skiprows=60, usecols=3, max_rows=1))
        values_charge.append(np.loadtxt(fname=txt_file, skiprows=61, usecols=8, max_rows=1))
        if txt_file[18:21] == 'CHL':
            ct.append(values_frag2_ufo[-1] - 2*values_frag1_ufo[-1] - values_pol[-1])
        if txt_file[18:21] != 'CHL':
            ct.append(2*values_frag2_ufo[-1] - values_frag1_ufo[-1] - values_pol[-1])

In [ ]:
fig, ax = plt.subplots(1,1)
ct = np.array(ct)*0.02
number_waters = []
for i in n_waters:
    if len(i)==2:
        n = 10*i[0]+i[1]
    else:
        n = i[0]   
    number_waters.append(n) 

CHL_data = []
POPC_data = []
popc_data = []
POPS_data = []
POPI_data = []
PSM_data = []
POGL_data = []
POPE_data = []

for idx, lipid_name in enumerate(lipid_names):
    if 'CHL' in lipid_name:
        ax.scatter(number_waters[idx], ct[idx], color='red')
        CHL_data.append((number_waters[idx], ct[idx]))
    if 'POPC' in lipid_name or "popc" in lipid_name:
        ax.scatter(number_waters[idx], ct[idx], color='black')
        POPC_data.append((number_waters[idx], ct[idx]))
        popc_data.append((number_waters[idx], ct[idx]))
    if 'POPE' in lipid_name:
        ax.scatter(number_waters[idx], ct[idx], color='cyan')
        POPE_data.append((number_waters[idx], ct[idx]))
    if 'POPS' in lipid_name:
        ax.scatter(number_waters[idx], ct[idx], color='green')
        POPS_data.append((number_waters[idx], ct[idx]))
    if 'POPI' in lipid_name:
        ax.scatter(number_waters[idx], ct[idx], color='blue')
        POPI_data.append((number_waters[idx], ct[idx]))
    if 'PSM' in lipid_name:
        ax.scatter(number_waters[idx], ct[idx], color='pink')
        PSM_data.append((number_waters[idx], ct[idx]))
    if 'POGL' in lipid_name:
        ax.scatter(number_waters[idx], ct[idx], color='purple')
        POGL_data.append((number_waters[idx], ct[idx]))
      


In [ ]:
POGL_data


In [ ]:

ax.scatter(number_waters, ct)